<a href="https://colab.research.google.com/github/shashidharbabu/IBMxTechXchange/blob/main/examples/notebooks/pdf-processing-1/pdf_processing_1_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing PDFs using Data Prep Kit

This notebook will introduce DPK and showcase some of it's capabilities.

Here is the workflow:

- pdf2parquet: Extract text from PDF documents
- docid: compute hashes
- exact dedupe : filter out identical documents
- fuzzy dedupe : filter out 'near duplicates'
- document quality: scoring documents for quality

![](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/images/data-prep-kit-3-workflow.png)


## How to run this notebook

Two options:

- **Option 1 - Google Colab:** easiest option.  no setup required.  Click this link to open this on google colab.  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sujee/data-prep-kit/blob/process-pdf-1/examples/notebooks/pdf-processing-1/pdf_processing_1_python.ipynb)
- **Option 2 - Local python dev environment:**  Setup using this [guide](../../../README.md#-getting-started)

The notebook will work as in both environments

## Step-1: Figure out Runtime Environment

### 1.1 - Determine runtime

Determine if we are running on Google colab or local python environment

In [1]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

Running in Colab


### 1.2 - Install dependencies if running on Google Colab

In [10]:
%%capture

if RUNNING_IN_COLAB:
    ! pip install  --default-timeout=100  \
        data-prep-toolkit-transforms[all]==1.0.0 \
        humanfriendly

In [6]:
!pip install humanfriendly

  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)


### 1.3 - Restart Runtime

After installing dependencies, be sure <font color="red">restart runtime</font>, so libraries will be loaded

You do this by going to **`Runtime --> Restart Session`**

Then you can continue to the next step (no need to re-run the notebook)

## Step-2: Configuration  & Utils

### 2.1 - Basic Config

In [3]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
   print("Running in Colab")
   RUNNING_IN_COLAB = True
else:
   print("NOT in Colab")
   RUNNING_IN_COLAB = False

Running in Colab


### 2.2 - Setup input/outpur directories

In [4]:
import os, sys
import shutil

input_dir = "input"
shutil.os.makedirs(input_dir, exist_ok=True)
output_dir = "output"

output_text_dir = os.path.join (output_dir, '01_text_out')
output_docid_dir = os.path.join (output_dir, '02_docid_out')
output_exact_dedupe_dir = os.path.join (output_dir, '03_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (output_dir, '04_fuzzy_dedupe_out')
output_doc_quality_dir = os.path.join (output_dir, '05_doc_quality_out')
output_final_dir = os.path.join (output_dir, 'output_final')

## clear output folder
shutil.rmtree(output_dir, ignore_errors=True)
shutil.os.makedirs(output_dir, exist_ok=True)
print ("✅ Cleared output directory")

✅ Cleared output directory


### 2.3 - Handy Utils

In [7]:
import os
import requests
from humanfriendly import format_size
import pandas as pd
import glob

## Reads parquet files in a folder into a pandas dataframe
def read_parquet_files_as_df (parquet_dir):
    parquet_files = glob.glob(f'{parquet_dir}/*.parquet')
    # read each parquet file into a DataFrame and store in a list
    dfs = [pd.read_parquet (f) for f in parquet_files]
    dfs = [df for df in dfs if not df.empty]  # filter out empty dataframes
    # Concatenate all DataFrames into a single DataFrame
    if len(dfs) > 0:
        data_df = pd.concat(dfs, ignore_index=True)
        return data_df
    else:
        return pd.DataFrame() # return empty df
# ------------


def download_file(url, local_file, chunk_size=1024*1024):
    """
    Downloads a remote URL to a local file.

    Args:
        url (str): The remote URL.
        local_filename (str): The name of the local file to save the downloaded content.
        chunk_size (int): The size in bytes of each chunk. Defaults to 1024.

    Returns:
        None

    Example usage:
        download_file('http://example.com/file.txt', 'file.txt', chunk_size=1024*1024)  # Download in chunks of 1MB
    """
    # Check if the local file already exists
    if os.path.exists(local_file):
        file_size = format_size(os.path.getsize(local_file))
        print(f"Local file '{local_file}' ({file_size}) already exists. Skipping download.")
        return

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(local_file), exist_ok=True)

    # Stream the file download
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_file, 'wb') as f:
            for chunk in r.iter_content(chunk_size=chunk_size):
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        print()
        file_size = format_size(os.path.getsize(local_file))
        print(f"{local_file} ({file_size}) downloaded successfully.")
## --- end: download_file ------



## Step-3: Inspect the Data

We will use simple PDFs.  The files are [here](https://github.com/IBM/data-prep-kit/tree/dev/examples/notebooks/pdf-processing-1/input/)

- [earth.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth.pdf) and exact duplicate [earth-copy.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth-copy.pdf)
- [earth2.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth2.pdf) almost similar to earth.pdf (ONE word difference!)
- [mars.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/mars.pdf)
- [spam.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/spam.pdf) - contains spammy contents
- [lorem-ipsum.pdf](https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/lorem-ipsum.pdf) - contains 'lorem ipsum' placeholder


### 3.1 -Download Data

In [8]:

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth.pdf', os.path.join(input_dir, 'earth.pdf'))

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth-copy.pdf', os.path.join(input_dir, 'earth-copy.pdf'))

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/earth2.pdf', os.path.join(input_dir, 'earth2.pdf'))

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/mars.pdf', os.path.join(input_dir, 'mars.pdf'))

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/spam.pdf', os.path.join(input_dir, 'spam.pdf'))

download_file ('https://raw.githubusercontent.com/sujee/data-prep-kit/process-pdf-1/examples/notebooks/pdf-processing-1/input/lorem-ipsum.pdf', os.path.join(input_dir, 'lorem-ipsum.pdf'))


input/earth.pdf (58.53 KB) downloaded successfully.

input/earth-copy.pdf (58.53 KB) downloaded successfully.

input/earth2.pdf (58.53 KB) downloaded successfully.

input/mars.pdf (57.87 KB) downloaded successfully.

input/spam.pdf (24.87 KB) downloaded successfully.

input/lorem-ipsum.pdf (25.72 KB) downloaded successfully.


## Step-4: Extract Data from PDF (pdf2parquet)

This step we will read PDF files and extract the text data.

[Pdf2Parquet documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/pdf2parquet/README.md)

We use the [Docling package](https://github.com/DS4SD/docling).


### 4.1 - Execute

In [11]:
%%time

from dpk_pdf2parquet.transform_python import Pdf2Parquet
from dpk_pdf2parquet.transform import pdf2parquet_contents_types

STAGE = 1
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_dir}' --> output='{output_text_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= input_dir,
                    output_folder= output_text_dir,
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,   # markdown
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_text_out'



19:04:26 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
INFO:dpk_pdf2parquet.transform:pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
19:04:26 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
19:04:26 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code locat

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

.gitignore:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

...

(…)del_artifacts/tableformer/tm_config.json:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

19:05:19 INFO - Completed 1 files (16.67%) in 0.07 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 1 files (16.67%) in 0.07 min
19:05:23 INFO - Completed 2 files (33.33%) in 0.129 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 2 files (33.33%) in 0.129 min
19:05:26 INFO - Completed 3 files (50.0%) in 0.188 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 3 files (50.0%) in 0.188 min
19:05:30 INFO - Completed 4 files (66.67%) in 0.253 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 4 files (66.67%) in 0.253 min
19:05:33 INFO - Completed 5 files (83.33%) in 0.303 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 5 files (83.33%) in 0.303 min
19:05:37 INFO - Completed 6 files (100.0%) in 0.363 min
INFO:data_processing.runtime.pure_python.transform_orchestrator:Completed 6 files (100.0%) in 0.363 min
19:05:37 INFO - Done processing 6 files, wai

✅ Stage:1 completed successfully
CPU times: user 57.2 s, sys: 7.96 s, total: 1min 5s
Wall time: 1min 27s


### 4.2 - Inspect Generated output

Here we should see one entry per input file processed.

In [12]:
print ("Displaying contents of : ", output_text_dir)
output_df = read_parquet_files_as_df(output_text_dir)
# print ("Output dimensions (rows x columns)= ", output_df.shape)
output_df.head()

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Displaying contents of :  output/01_text_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-23T19:05:33.860016,2.982996,mars.pdf
1,spam.pdf,Free xxx,1,0,2,1c58b5df-4dc6-44ce-87a2-98031c70212e,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-23T19:05:37.432981,3.564208,spam.pdf
2,lorem-ipsum.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,3f0532e6-4b97-492f-b284-a729510d3876,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-23T19:05:30.866883,3.895044,lorem-ipsum.pdf
3,earth.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,645c0708-a325-4a39-ab8d-622a8b4fc03f,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:23.399282,3.555579,earth.pdf
4,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:19.828861,4.166960,earth-copy.pdf



### 4.3 - Understand the output

Here are some interesting attributes to note:

- **filename** : original filename
- **contents** : text
- **document_id**: unique id (UUID) assignd to this document
- **document_hash**: hash of documents
- **hash** : hash of `contents` column
- **pdf_convert_time** : time to convert this pdf in seconds

**Note: you should notice the hash values are identical for the duplicate documents**

Let's inspect the **contents** column.

In [13]:
print (output_df.iloc[0, ]['contents'])

## Mars

## Solar System

Our solar system is a vast and fascinating expanse, comprising eight planets, ﬁve dwarf planets, numerous moons, asteroids, comets, and other celestial bodies. At its center lies the star we call the Sun.

For more details about the Solar system see Chapter 1.

## Mars

Mars, the fourth planet from the Sun, is a cold, desert world with a thin atmosphere composed primarily of carbon dioxide. Its reddish hue comes from iron oxide, or rust, prevalent on its surface.

## Basic facts about Mars:

- · Distance from the Sun: Average of 228 million kilometers (142 million miles)
- · Moons: Two small moons, Phobos and Deimos.
- · Rotation Period: 24.6 hours (one Martian day - called a "sol")


In [14]:
print (output_df.iloc[1, ]['contents'])


Free xxx


## Step-5:  Create DOC ID for Documents

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set **hash_column** to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set **int_id_column** to the name of the column, where you want to store it.

**This step is a pre-requisite for fuzzy dedup** in the pipeline.

[DocID documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/doc_id)

### 5.1 - Execute

In [15]:
%%time

from dpk_doc_id.transform_python import DocID

STAGE = 2
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_text_dir}' --> output='{output_docid_dir}'\n", flush=True)

result = DocID(input_folder= output_text_dir,
        output_folder= output_docid_dir,
        doc_id_doc_column= "contents",
        doc_id_hash_column= "doc_hash",
        # doc_id_int_column= "doc_id",
        doc_id_int_column= "int_id_column",
        #doc_id_start_id= 5
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")


🏃🏼 STAGE-2: Processing input='output/01_text_out' --> output='output/02_docid_out'



19:07:17 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'doc_hash', 'int_column': 'int_id_column', 'start_id': 0}
INFO:dpk_doc_id.transform:Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'doc_hash', 'int_column': 'int_id_column', 'start_id': 0}
19:07:17 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
19:07:17 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
19:07:17 INFO - data factory data_ is using local data access: input_folder - output/01_text_out output_folder - output/02_docid_out
INFO:data_processing.data_access.data_access_factory_base19be282f-5196-4a8e-83b6-b31a76ab02bd:data factory data_ is using local data access: input_folder - output/01_text_out output_folder - output/02_docid_out
19:07:17 INFO - data factory data_ max_files -1, n_sample -1
INFO:data_processing.data_access.data_access_factory_base19be282f-5196-4a8e-83b6-

✅ Stage:2 completed successfully
CPU times: user 136 ms, sys: 7.63 ms, total: 144 ms
Wall time: 200 ms


### 5.2 - Inspect Generated output

You would see a new columns **doc_hash** and **int_id_column**

In [16]:
print ("Displaying contents of : ", output_docid_dir)
output_df = read_parquet_files_as_df(output_docid_dir)
output_df.head()

Displaying contents of :  output/02_docid_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-23T19:05:33.860016,2.982996,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4
1,spam.pdf,Free xxx,1,0,2,1c58b5df-4dc6-44ce-87a2-98031c70212e,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-23T19:05:37.432981,3.564208,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5
2,lorem-ipsum.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,3f0532e6-4b97-492f-b284-a729510d3876,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-23T19:05:30.866883,3.895044,lorem-ipsum.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3
3,earth.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,645c0708-a325-4a39-ab8d-622a8b4fc03f,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:23.399282,3.555579,earth.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,1
4,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:19.828861,4.166960,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0


## Step-6: Eliminate Duplicate Documents

We have 2 exact duplicates: **earth.pdf** , **earth-copy.pdf**

Note how **doc_hash** for these documents are the same.

[Exact dedupe information](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/ededup)

### 6.1 - Execute

In [17]:
%%time

from dpk_ededup.transform_python import Ededup

STAGE = 3
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_docid_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_docid_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="doc_hash"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_docid_out' --> output='output/03_exact_dedupe_out'



19:07:52 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'doc_hash', 'use_snapshot': False, 'snapshot_directory': None}
INFO:dpk_ededup.transform_base:exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'doc_hash', 'use_snapshot': False, 'snapshot_directory': None}
19:07:52 INFO - pipeline id pipeline_id
INFO:data_processing.runtime.execution_configuration:pipeline id pipeline_id
19:07:52 INFO - code location None
INFO:data_processing.runtime.execution_configuration:code location None
19:07:52 INFO - data factory data_ is using local data access: input_folder - output/02_docid_out output_folder - output/03_exact_dedupe_out
INFO:data_processing.data_access.data_access_factory_base19be282f-5196-4a8e-83b6-b31a76ab02bd:data factory data_ is using local data access: input_folder - output/02_docid_out output_folder - output/03_exact_dedupe_out
19:07:52 INFO - data factory data_ max_files -1, n_sample -1
INFO:data_processing.data_access.data_access_fac

✅ Stage:3 completed successfully
CPU times: user 116 ms, sys: 8.66 ms, total: 125 ms
Wall time: 151 ms


### 6.2 - Inspect Generated output

You can see one of **earth.pdf** or **earth-copy.pdf** will be eliminated.

In [18]:
input_df = read_parquet_files_as_df(output_docid_dir)
output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

print ("Displaying contents of : ", output_exact_dedupe_dir)
output_df.head()

Input files before exact dedupe : 6
Output files after exact dedupe : 5
Duplicate files removed :   1
Displaying contents of :  output/03_exact_dedupe_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column,removed
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-23T19:05:33.860016,2.982996,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4,[]
1,spam.pdf,Free xxx,1,0,2,1c58b5df-4dc6-44ce-87a2-98031c70212e,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-23T19:05:37.432981,3.564208,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5,[]
2,lorem-ipsum.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,3f0532e6-4b97-492f-b284-a729510d3876,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-23T19:05:30.866883,3.895044,lorem-ipsum.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3,[]
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:19.828861,4.166960,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0,[]
4,earth2.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,487b2459-c0a8-4e46-af88-ed86563dd779,10729312978404042321,pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,610,2025-01-23T19:05:26.955981,3.547894,earth2.pdf,f039191d59ce8ba25023a844f9b99e7ef2ea4bf75a23f4...,2,[]


## Step-7: Fuzzy Dedupe

In previous step, we removed **exact duplicates (identical documents)**.

Fuzzy de-dupe can further filter out documents that are **not exactly identical, but nearly identical**

Here is a simple example:

`Our solar system is a vast and fascinating expanse`

`The solar system is a vast and fascinating expanse`

Only one word is different `Our` vs `The`.

Imagine two documents with one extra blank line.  For our purposes they are the same.

[Fuzzy dedupe documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/universal/fdedup)

### Tweaking fuzzy matches

**`jaccard_similarity_threshold`** is the parameter used to tweak similarities between documents.  It's value is between 0 and 1.0.  Values close to 1.0 means more strict checking (fewer documents will qualify).  Lower threshold means more leniant matches (more documents will qualify)

Adjust this value to find what works for your documents

### 7.1 - Execute

In [19]:
%%time

from dpk_fdedup.transform_python import Fdedup

STAGE = 4
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_fuzzy_dedupe_dir}'\n", flush=True)

result = Fdedup(input_folder=output_exact_dedupe_dir,
                output_folder=output_fuzzy_dedupe_dir,
                contents_column= "contents",
                # document_id_column= "doc_id",
                document_id_column= "int_id_column",
                num_permutations= 112,
                num_bands= 14,
                num_minhashes_per_band= 8,
                jaccard_similarity_threshold = 0.8, # between 0 - 1.  higher means more strict checking
                operation_mode="filter_duplicates",
                # operation_mode="annotate",
                ).transform()
# if result == 0:
#     print (f"✅ Stage:{STAGE} completed successfully")
# else:
#     raise Exception (f"❌ Stage:{STAGE}  failed (result={result})")

🏃🏼 STAGE-4: Processing input='output/03_exact_dedupe_out' --> output='output/04_fuzzy_dedupe_out'



19:08:58 INFO - Starting SignatureCalculation step
INFO:dpk_fdedup.transform_python:Starting SignatureCalculation step
19:08:58 INFO - Got parameters for SignatureCalculation
INFO:dpk_fdedup.transform_python:Got parameters for SignatureCalculation
19:08:58 INFO - minhash parameters are : {'document_id_column': 'int_id_column', 'contents_column': 'contents', 'seed': 42, 'num_permutations': 112, 'jaccard_similarity_threshold': 0.8, 'word_shingle_size': 5, 'num_bands': 14, 'num_minhashes_per_band': 8, 'num_segments': 1, 'shingle_option': 'word'}
INFO:dpk_fdedup.signature_calc.transform:minhash parameters are : {'document_id_column': 'int_id_column', 'contents_column': 'contents', 'seed': 42, 'num_permutations': 112, 'jaccard_similarity_threshold': 0.8, 'word_shingle_size': 5, 'num_bands': 14, 'num_minhashes_per_band': 8, 'num_segments': 1, 'shingle_option': 'word'}
19:08:58 INFO - data factory scdata_ is using local configuration without input/output path
INFO:data_processing.data_access.

CPU times: user 1.08 s, sys: 129 ms, total: 1.21 s
Wall time: 1.91 s


### 7.2 - Inspect Output

FuzzyDedupe will write documents that are filtered in **output/04_fuzzy_dedupe_out/cleaned** folder

You will notice only one **earth.pdf** made it!  So fuzzy dedupe did filter out the almost identical doc.

In [20]:
input_df = read_parquet_files_as_df(output_exact_dedupe_dir)
output_df = read_parquet_files_as_df(os.path.join(output_fuzzy_dedupe_dir, "cleaned"))

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Near duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

print ("Displaying contents of : ", output_fuzzy_dedupe_dir)
output_df.head()

Input files before exact dedupe : 5
Output files after exact dedupe : 4
Near duplicate files removed :   1
Displaying contents of :  output/04_fuzzy_dedupe_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,doc_hash,int_id_column,removed
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,2025-01-23T19:05:33.860016,2.982996,mars.pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,4,[]
1,spam.pdf,Free xxx,1,0,2,1c58b5df-4dc6-44ce-87a2-98031c70212e,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,2025-01-23T19:05:37.432981,3.564208,spam.pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,5,[]
2,lorem-ipsum.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,3f0532e6-4b97-492f-b284-a729510d3876,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,2025-01-23T19:05:30.866883,3.895044,lorem-ipsum.pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,3,[]
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,2025-01-23T19:05:19.828861,4.166960,earth-copy.pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,0,[]


## Step-8: Document Quality

This handy plugin will score documents across many metrics.

Here we will look for 'bad words' metric.

[Document quality documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/doc_quality)

By default it uses [bad words collection](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/doc_quality/dpk_doc_quality/ldnoobw).  You can supply a custom file by passing an argument `bad_word_filepath=/path/to/badwords_file`

### 8.1 - Execute

In [21]:
%%time

from dpk_doc_quality.transform_python import DocQuality

STAGE = 5
output_fuzzy_dedupe_cleaned_dir = os.path.join(output_fuzzy_dedupe_dir, "cleaned")
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_fuzzy_dedupe_cleaned_dir}' --> output='{output_doc_quality_dir}'\n", flush=True)

result = DocQuality(input_folder=output_fuzzy_dedupe_cleaned_dir,
                    output_folder= output_doc_quality_dir,
                    docq_text_lang = "en",
                    docq_doc_content_column ="contents",
                    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed (result={result})")

🏃🏼 STAGE-5: Processing input='output/04_fuzzy_dedupe_out/cleaned' --> output='output/05_doc_quality_out'



19:09:33 INFO - doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/usr/local/lib/python3.11/dist-packages/dpk_doc_quality/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x7b391d139b10>}
INFO:dpk_doc_quality.transform:doc_quality parameters are : {'text_lang': 'en', 'doc_content_column': 'contents', 'bad_word_filepath': '/usr/local/lib/python3.11/dist-packages/dpk_doc_quality/ldnoobw/en', 's3_cred': None, 'docq_data_factory': <data_processing.data_access.data_access_factory.DataAccessFactory object at 0x7b391d139b10>}
19:09:33 INFO - data factory docq_ is using local configuration without input/output path
INFO:data_processing.data_access.data_access_factory_basea9bf74c0-2c16-4dac-95b3-3004d5a45699:data factory docq_ is using local configuration without input/output path
19:09:33 INFO - data factory docq_ max_files -1, n_sample -1
INFO:data_processing.d

✅ Stage:5 completed successfully
CPU times: user 139 ms, sys: 15 ms, total: 154 ms
Wall time: 158 ms


### 8.2 - Inspect the Output

We will see several new columns starting with the name **docq_**.

We will look at a metric **docq_contain_bad_word** and filter out any documents that have bad words.

For more information see : [Doc Quality documentation](https://github.com/IBM/data-prep-kit/tree/dev/transforms/language/doc_quality)

In [22]:
output_df = read_parquet_files_as_df(output_doc_quality_dir)
print ("Displaying contents of : ", output_doc_quality_dir)
output_df.head()

Displaying contents of :  output/05_doc_quality_out


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,...,4.688000,0.032000,8,0.000000,0.0,False,0.176471,0.0,0.880000,True
1,spam.pdf,Free xxx,1,0,2,1c58b5df-4dc6-44ce-87a2-98031c70212e,10026122586747302274,pdf,543ffc97aef373ee009a5f908e0358ef80d329ca7ba964...,8,...,3.500000,0.000000,1,0.000000,0.0,True,0.000000,0.0,1.000000,False
2,lorem-ipsum.pdf,Lorem ipsum Lorem ipsum Lorem ipsum,1,0,2,3f0532e6-4b97-492f-b284-a729510d3876,6571294142213095721,pdf,bc012d063005cc02deb6c2592d1f8c3b273625edf9eec5...,35,...,5.000000,0.000000,1,0.085714,0.0,False,0.000000,0.0,1.000000,False
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,...,4.541284,0.027523,9,0.000000,0.0,False,0.176471,0.0,0.880734,True


### 8.3 - Filtering 'quality' documents

So from the output above we see **spam.pdf** is flagged for containing bad words (**docq_contain_bad_word=True**).

Also **lorem.pdf** is flagged for place holder content **lorem ipsum**  (**docq_lorem_ipsum_ratio > 0**)

We are going to filter them both out

In [23]:
all_docs_df = read_parquet_files_as_df(output_doc_quality_dir)

# remove documents with badwords
clean_docs_df = all_docs_df[all_docs_df['docq_contain_bad_word'] == False]

# also filter out 'lorem ipsum' text
clean_docs_df = clean_docs_df[clean_docs_df['docq_lorem_ipsum_ratio'] == 0]

clean_docs_df.head()

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,...,docq_mean_word_len,docq_symbol_to_word_ratio,docq_sentence_count,docq_lorem_ipsum_ratio,docq_curly_bracket_ratio,docq_contain_bad_word,docq_bullet_point_ratio,docq_ellipsis_line_ratio,docq_alphabet_word_ratio,docq_contain_common_en_words
0,mars.pdf,## Mars\n\n## Solar System\n\nOur solar system...,1,0,11,1fc9cc53-0d2d-4a8e-8f70-ff36caa14fb2,7758129997476962679,pdf,a3a4bb3b8f4f441d6d669e09f0cd07a9420d06850cf63e...,717,...,4.688000,0.032000,8,0.0,0.0,False,0.176471,0.0,0.880000,True
3,earth-copy.pdf,## Earth\n\n## Solar System\n\nOur solar syste...,1,0,11,84195391-9791-47bf-9fef-c2c3db786085,14711865278795535908,pdf,6140cf695f269a3ddca6568536076756105ad3186086b2...,610,...,4.541284,0.027523,9,0.0,0.0,False,0.176471,0.0,0.880734,True


## Step-9: Copy output to final output dir

In [24]:
import shutil

shutil.rmtree(output_final_dir, ignore_errors=True)
shutil.os.makedirs(output_final_dir, exist_ok=True)

output_final_dir_parquet = os.path.join (output_final_dir, 'pq')
shutil.os.makedirs(output_final_dir_parquet, exist_ok=True)

output_final_dir_markdown = os.path.join (output_final_dir, 'markdown')
shutil.os.makedirs(output_final_dir_markdown, exist_ok=True)

In [25]:
## save parquet

clean_docs_df.to_parquet(os.path.join(output_final_dir_parquet, "clean_docs.parquet"))
print (f"✅ Saved CLEAN parquet output to '{output_final_dir_parquet}'")

✅ Saved CLEAN parquet output to 'output/output_final/pq'


In [26]:
## save markdown text

for index, row in clean_docs_df.iterrows():
    output_file_name = os.path.join (output_final_dir_markdown, row['filename'] + '.md')
    with open(output_file_name, 'w') as output_file:
        output_file.write(row['contents'])

print (f"✅ Saved CLEAN markdown output to '{output_final_dir_markdown}'")


✅ Saved CLEAN markdown output to 'output/output_final/markdown'


In [28]:
print(output_final_dir_markdown)

output/output_final/markdown
